# Addition of a Neural Network Model

After creating the previous three models for consideration, I decided to go back and take a look at the possibility of adding a fourth model. I wanted to see if I can use a model with Deep Neural Networks for a more realistic assessment of housing prices. I was able to reach my goal of coming under $200000, but I believe that I can do even better with the use of a neural network. I am going to bring in my cleaned data so that I can go right into the modeling process. 

# Importing Libraries

In [1]:
import tensorflow as tf
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
np.random.seed(9)
random.seed(9)
tf.random.set_seed(9)

# Bringing in the Cleaned Dataset

In [3]:
#bringing in the cleaned
infile = open("Data/cleaned_data.pickle",'rb')
df = pickle.load(infile)
infile.close()

In [4]:
df.head()

,price,bedrooms,bathrooms,sqft_living,floors,waterfront,sqft_above,sqft_basement,sqft_living15,log_yard,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,365000.0,4.0,2.0,2070.0,2.0,0,2070.0,0,2390,8.969287,...,0,0,0,0,0,0,0,0,0,0
1,865000.0,5.0,3.0,2900.0,1.0,0,1830.0,1070,2370,8.250620,...,0,0,0,0,0,0,0,0,0,0
2,1038000.0,4.0,2.0,3770.0,2.0,0,3770.0,0,3710,9.105868,...,0,0,0,0,0,0,0,0,0,0
3,1490000.0,3.0,4.0,4560.0,2.0,0,4560.0,0,4050,9.419628,...,0,0,0,0,0,0,0,0,0,0
4,711000.0,3.0,2.0,2550.0,2.0,0,2550.0,0,2250,8.318986,...,0,0,0,0,0,0,0,0,0,0


# Preparing the Data

In [5]:
#dropping the dependent variable
dvariables = df.iloc[:, 1:]

#this here will become the SHAPE variable in the function below
number_of_columns = len(dvariables.columns)

#isolating the target variable
target = df.iloc[:,0]

In [6]:
#implementing train test split
X_fulltrain, X_fulltest, y_fulltrain, y_fulltest = train_test_split(dvariables, target, random_state = 9, test_size = .2)

#splitting again for a validation set
X_train, X_valid, y_train, y_valid = train_test_split(X_fulltrain, y_fulltrain, random_state = 9, test_size = .2)


# Model Creation

In [7]:
def create_model(optimizer = 'adam', num_layers = 1, activation = 'relu', neurons = 30, drop_out= .2, input_shape = 106, learning_rate = .003):
    """This function takes an optimizer, number of layers, and input_shape. It will create
    and compile the model so that it can be ready to be fit in the next step."""
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape = (input_shape,)))
    for layer in range(num_layers):
        model.add(tf.keras.layers.Dense(neurons, activation = activation))
    #This dropout should hopefully be preventing overfitting
    model.add(tf.keras.layers.Dropout(drop_out))
    model.add(tf.keras.layers.Activation(activation))
    model.add(tf.keras.layers.Dense(1))
    
    
    model.compile(optimizer, loss = 'mse', metrics = ['mse'])
    return model

In [8]:
model_1 = create_model(num_layers = 1)

In [9]:
#fitting the model
history = model_1.fit(X_train, y_train, epochs = 80, 
                      #utilizing our validation set to test/prevent overfitting
                      validation_data = (X_valid, y_valid)) 



Epoch 1/80
346/346 [==============================] - 0s 977us/step - loss: 427578458112.0000 - mse: 427578458112.0000 - val_loss: 383302434816.0000 - val_mse: 383302434816.0000
Epoch 2/80
346/346 [==============================] - 0s 655us/step - loss: 377481003008.0000 - mse: 377481003008.0000 - val_loss: 316028649472.0000 - val_mse: 316028649472.0000
Epoch 3/80
346/346 [==============================] - 0s 667us/step - loss: 297875111936.0000 - mse: 297875111936.0000 - val_loss: 232643461120.0000 - val_mse: 232643461120.0000
Epoch 4/80
346/346 [==============================] - 0s 659us/step - loss: 215591829504.0000 - mse: 215591829504.0000 - val_loss: 157850714112.0000 - val_mse: 157850714112.0000
Epoch 5/80
346/346 [==============================] - 0s 647us/step - loss: 150588375040.0000 - mse: 150588375040.0000 - val_loss: 106434174976.0000 - val_mse: 106434174976.0000
Epoch 6/80
346/346 [==============================] - 0s 652us/step - loss: 108495872000.0000 - mse: 108495872

Epoch 48/80
346/346 [==============================] - 0s 643us/step - loss: 79700746240.0000 - mse: 79700746240.0000 - val_loss: 65656176640.0000 - val_mse: 65656176640.0000
Epoch 49/80
346/346 [==============================] - 0s 648us/step - loss: 80272236544.0000 - mse: 80272236544.0000 - val_loss: 65657122816.0000 - val_mse: 65657122816.0000
Epoch 50/80
346/346 [==============================] - 0s 646us/step - loss: 81593925632.0000 - mse: 81593925632.0000 - val_loss: 65659469824.0000 - val_mse: 65659469824.0000
Epoch 51/80
346/346 [==============================] - 0s 680us/step - loss: 80642744320.0000 - mse: 80642744320.0000 - val_loss: 65666236416.0000 - val_mse: 65666236416.0000
Epoch 52/80
346/346 [==============================] - 0s 676us/step - loss: 79390244864.0000 - mse: 79390244864.0000 - val_loss: 65681113088.0000 - val_mse: 65681113088.0000
Epoch 53/80
346/346 [==============================] - 0s 662us/step - loss: 81966424064.0000 - mse: 81966424064.0000 - val_l

# Evaluation

In [10]:
print('Evaluation at Epoch 80:\n')
for key in history.history:
    print(key + ':' + str(history.history[key][-1]) + '\n')

print('RMSE :' + str(np.sqrt(history.history['val_mse'][-1])))

Evaluation at Epoch 80:

loss:80479903744.0

mse:80479903744.0

val_loss:65825914880.0

val_mse:65825914880.0

RMSE :256565.6151552659


In terms of the RMSE the model has not crossed the threshold of the goal RMSE. But the good news is that there does not seem to be evidence of overfitting. I will continue on with the nn modeling process and try to lower the error by tinkering manually with the parameters.

# Second Model Attempt

For an attempt at improvement, I am going to try another model with more layers to see how that would compare to the first attempt. 

In [11]:
#I am increasing the number of layers in this model
model_2 = create_model(num_layers = 12)

In [12]:
history_2 = model_2.fit(X_train, y_train, epochs = 80, validation_data=(X_valid, y_valid))

Epoch 1/80
346/346 [==============================] - 0s 1ms/step - loss: 143858401280.0000 - mse: 143858401280.0000 - val_loss: 65858732032.0000 - val_mse: 65858732032.0000
Epoch 2/80
346/346 [==============================] - 0s 1ms/step - loss: 85689106432.0000 - mse: 85689106432.0000 - val_loss: 68905230336.0000 - val_mse: 68905230336.0000
Epoch 3/80
346/346 [==============================] - 0s 1ms/step - loss: 84045225984.0000 - mse: 84045225984.0000 - val_loss: 65371648000.0000 - val_mse: 65371648000.0000
Epoch 4/80
346/346 [==============================] - 0s 1ms/step - loss: 82223259648.0000 - mse: 82223259648.0000 - val_loss: 65959161856.0000 - val_mse: 65959161856.0000
Epoch 5/80
346/346 [==============================] - 0s 1ms/step - loss: 80372695040.0000 - mse: 80372695040.0000 - val_loss: 70882213888.0000 - val_mse: 70882213888.0000
Epoch 6/80
346/346 [==============================] - 0s 1ms/step - loss: 78023237632.0000 - mse: 78023237632.0000 - val_loss: 71993516032

346/346 [==============================] - 0s 1ms/step - loss: 59613036544.0000 - mse: 59613036544.0000 - val_loss: 41280327680.0000 - val_mse: 41280327680.0000
Epoch 49/80
346/346 [==============================] - 0s 1ms/step - loss: 55575535616.0000 - mse: 55575535616.0000 - val_loss: 40885018624.0000 - val_mse: 40885018624.0000
Epoch 50/80
346/346 [==============================] - 0s 1ms/step - loss: 56002883584.0000 - mse: 56002883584.0000 - val_loss: 41237901312.0000 - val_mse: 41237901312.0000
Epoch 51/80
346/346 [==============================] - 0s 1ms/step - loss: 54529191936.0000 - mse: 54529191936.0000 - val_loss: 37793935360.0000 - val_mse: 37793935360.0000
Epoch 52/80
346/346 [==============================] - 0s 1ms/step - loss: 54906830848.0000 - mse: 54906830848.0000 - val_loss: 48173109248.0000 - val_mse: 48173109248.0000
Epoch 53/80
346/346 [==============================] - 0s 1ms/step - loss: 55747051520.0000 - mse: 55747051520.0000 - val_loss: 39492460544.0000 - 

In [13]:
print('Evaluation at Epoch 80:\n')
for key in history_2.history:
    print(key + ':' + str(history_2.history[key][-1]) + '\n')
    
print('RMSE :' + str(np.sqrt(history_2.history['val_mse'][-1])))

Evaluation at Epoch 80:

loss:48876310528.0

mse:48876310528.0

val_loss:30303090688.0

val_mse:30303090688.0

RMSE :174077.8293982321


The error here is reduced from the last attempt. This tells us that more layers might have been needed to get a better estimate of the target. The error measured in the validation set gets us over the threshold and achieves our goal of less than 200000. It does not however compare to the original models created. I think the next best step would be to incorporate randomized search to find optimal parameters.

# Implementing RandomizedSearch

In [14]:
#create a base model for Gridsearch
model = KerasRegressor(build_fn=create_model)

In [15]:
neurons = tuple(range(20, 101))
drop_out = (.1, .2, .5, .8, .9)
epochs = (70, 80, 90, 100, 110)
num_layers = tuple(range(7, 19))
optimizer = ('Adam', 'RMSprop')
activation = ('relu', 'linear')
batch_size = tuple(range(10, 51))
parameters = {'neurons': neurons, 'drop_out': drop_out, 'num_layers': num_layers,
             'optimizer': optimizer, 'epochs':epochs, 'activation': activation}

In [16]:
random_model = RandomizedSearchCV(model, parameters, cv = 3, verbose = 1)

In [17]:
history_3 = random_model.fit(X_train, y_train, validation_data = (X_valid, y_valid),
                             #I'm going to try to use Early Stopping. It will hopefully
                             #get a lower score, and maybe will lower the total runtime.
                            callbacks = [tf.keras.callbacks.EarlyStopping(patience = 20)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/110


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


231/231 [==============================] - 0s 2ms/step - loss: 128026206208.0000 - mse: 128026206208.0000 - val_loss: 76087574528.0000 - val_mse: 76087574528.0000
Epoch 2/110
231/231 [==============================] - 0s 1ms/step - loss: 107023376384.0000 - mse: 107023376384.0000 - val_loss: 77711384576.0000 - val_mse: 77711384576.0000
Epoch 3/110
231/231 [==============================] - 0s 1ms/step - loss: 105187041280.0000 - mse: 105187041280.0000 - val_loss: 66956767232.0000 - val_mse: 66956767232.0000
Epoch 4/110
231/231 [==============================] - 0s 1ms/step - loss: 107576991744.0000 - mse: 107576991744.0000 - val_loss: 69781856256.0000 - val_mse: 69781856256.0000
Epoch 5/110
231/231 [==============================] - 0s 1ms/step - loss: 108987678720.0000 - mse: 108987678720.0000 - val_loss: 69714837504.0000 - val_mse: 69714837504.0000
Epoch 6/110
231/231 [==============================] - 0s 1ms/step - loss: 102607159296.0000 - mse: 102607159296.0000 - val_loss: 6898302

Epoch 48/110
231/231 [==============================] - 0s 1ms/step - loss: 84541390848.0000 - mse: 84541390848.0000 - val_loss: 56428843008.0000 - val_mse: 56428843008.0000
Epoch 49/110
231/231 [==============================] - 0s 1ms/step - loss: 89195487232.0000 - mse: 89195487232.0000 - val_loss: 48448937984.0000 - val_mse: 48448937984.0000
Epoch 50/110
231/231 [==============================] - 0s 1ms/step - loss: 86024011776.0000 - mse: 86024011776.0000 - val_loss: 55390265344.0000 - val_mse: 55390265344.0000
Epoch 51/110
231/231 [==============================] - 0s 1ms/step - loss: 94649540608.0000 - mse: 94649540608.0000 - val_loss: 51091496960.0000 - val_mse: 51091496960.0000
Epoch 52/110
231/231 [==============================] - 0s 1ms/step - loss: 92601597952.0000 - mse: 92601597952.0000 - val_loss: 48761708544.0000 - val_mse: 48761708544.0000
Epoch 53/110
231/231 [==============================] - 0s 1ms/step - loss: 88464605184.0000 - mse: 88464605184.0000 - val_loss: 5

231/231 [==============================] - 0s 1ms/step - loss: 76857180160.0000 - mse: 76857180160.0000 - val_loss: 38806417408.0000 - val_mse: 38806417408.0000
Epoch 96/110
231/231 [==============================] - 0s 1ms/step - loss: 73625034752.0000 - mse: 73625034752.0000 - val_loss: 36438908928.0000 - val_mse: 36438908928.0000
Epoch 97/110
231/231 [==============================] - 0s 1ms/step - loss: 69890949120.0000 - mse: 69890949120.0000 - val_loss: 36953247744.0000 - val_mse: 36953247744.0000
Epoch 98/110
231/231 [==============================] - 0s 1ms/step - loss: 71625768960.0000 - mse: 71625768960.0000 - val_loss: 34258722816.0000 - val_mse: 34258722816.0000
Epoch 99/110
231/231 [==============================] - 0s 1ms/step - loss: 71409623040.0000 - mse: 71409623040.0000 - val_loss: 35798056960.0000 - val_mse: 35798056960.0000
Epoch 100/110
231/231 [==============================] - 0s 1ms/step - loss: 71951982592.0000 - mse: 71951982592.0000 - val_loss: 31905802240.0

...

231/231 [==============================] - 0s 1ms/step - loss: 141009272832.0000 - mse: 141009272832.0000 - val_loss: 89667198976.0000 - val_mse: 89667198976.0000
Epoch 25/110
231/231 [==============================] - 0s 1ms/step - loss: 141504118784.0000 - mse: 141504118784.0000 - val_loss: 76270288896.0000 - val_mse: 76270288896.0000
Epoch 26/110
231/231 [==============================] - 0s 1ms/step - loss: 140326469632.0000 - mse: 140326469632.0000 - val_loss: 89511690240.0000 - val_mse: 89511690240.0000
Epoch 27/110
231/231 [==============================] - 0s 1ms/step - loss: 146738724864.0000 - mse: 146738724864.0000 - val_loss: 78489575424.0000 - val_mse: 78489575424.0000
Epoch 28/110
231/231 [==============================] - 0s 1ms/step - loss: 136687312896.0000 - mse: 136687312896.0000 - val_loss: 63989161984.0000 - val_mse: 63989161984.0000
Epoch 29/110
231/231 [==============================] - 0s 1ms/step - loss: 139980570624.0000 - mse: 139980570624.0000 - val_loss: 88

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 14.9min finished


346/346 [==============================] - 1s 2ms/step - loss: 94894833664.0000 - mse: 94894833664.0000 - val_loss: 66287689728.0000 - val_mse: 66287689728.0000
Epoch 2/100
346/346 [==============================] - 1s 2ms/step - loss: 82964316160.0000 - mse: 82964316160.0000 - val_loss: 69508521984.0000 - val_mse: 69508521984.0000
Epoch 3/100
346/346 [==============================] - 1s 2ms/step - loss: 81064026112.0000 - mse: 81064026112.0000 - val_loss: 66379784192.0000 - val_mse: 66379784192.0000
Epoch 4/100
346/346 [==============================] - 1s 1ms/step - loss: 82943606784.0000 - mse: 82943606784.0000 - val_loss: 66791829504.0000 - val_mse: 66791829504.0000
Epoch 5/100
346/346 [==============================] - 1s 2ms/step - loss: 78457126912.0000 - mse: 78457126912.0000 - val_loss: 65931177984.0000 - val_mse: 65931177984.0000
Epoch 6/100
346/346 [==============================] - 0s 1ms/step - loss: 78846197760.0000 - mse: 78846197760.0000 - val_loss: 79729213440.0000 - 

346/346 [==============================] - 1s 1ms/step - loss: 47371833344.0000 - mse: 47371833344.0000 - val_loss: 33641611264.0000 - val_mse: 33641611264.0000
Epoch 49/100
346/346 [==============================] - 1s 2ms/step - loss: 45583114240.0000 - mse: 45583114240.0000 - val_loss: 34490208256.0000 - val_mse: 34490208256.0000
Epoch 50/100
346/346 [==============================] - 1s 2ms/step - loss: 48846450688.0000 - mse: 48846450688.0000 - val_loss: 30665545728.0000 - val_mse: 30665545728.0000
Epoch 51/100
346/346 [==============================] - 1s 2ms/step - loss: 46563303424.0000 - mse: 46563303424.0000 - val_loss: 34109677568.0000 - val_mse: 34109677568.0000
Epoch 52/100
346/346 [==============================] - 1s 1ms/step - loss: 45338152960.0000 - mse: 45338152960.0000 - val_loss: 37027405824.0000 - val_mse: 37027405824.0000
Epoch 53/100
346/346 [==============================] - 1s 1ms/step - loss: 45060243456.0000 - mse: 45060243456.0000 - val_loss: 33844510720.00

346/346 [==============================] - 1s 2ms/step - loss: 41734082560.0000 - mse: 41734082560.0000 - val_loss: 32449595392.0000 - val_mse: 32449595392.0000
Epoch 96/100
346/346 [==============================] - 1s 2ms/step - loss: 41792827392.0000 - mse: 41792827392.0000 - val_loss: 30966396928.0000 - val_mse: 30966396928.0000
Epoch 97/100
346/346 [==============================] - 0s 1ms/step - loss: 39850360832.0000 - mse: 39850360832.0000 - val_loss: 26231431168.0000 - val_mse: 26231431168.0000
Epoch 98/100
346/346 [==============================] - 1s 2ms/step - loss: 43165380608.0000 - mse: 43165380608.0000 - val_loss: 31653451776.0000 - val_mse: 31653451776.0000
Epoch 99/100
346/346 [==============================] - 1s 2ms/step - loss: 38136762368.0000 - mse: 38136762368.0000 - val_loss: 85482168320.0000 - val_mse: 85482168320.0000
Epoch 100/100
346/346 [==============================] - 1s 2ms/step - loss: 40367177728.0000 - mse: 40367177728.0000 - val_loss: 28165097472.0

In [18]:
#getting the best parameters
history_3.best_params_

{'optimizer': 'Adam',
 'num_layers': 14,
 'neurons': 82,
 'epochs': 100,
 'drop_out': 0.1,
 'activation': 'linear'}

In [19]:
#checking the RMSE
np.sqrt(abs(history_3.best_score_))

196106.94834536928

In [20]:
model_3 = create_model(optimizer = 'Adam', num_layers = 14, neurons = 82, drop_out = .1, activation = 'linear')

In [21]:
history_3 = model_3.fit(X_train, y_train, epochs = 100, validation_data=(X_valid, y_valid),
                       callbacks = [tf.keras.callbacks.EarlyStopping(patience = 20)])

Epoch 1/100
346/346 [==============================] - 1s 2ms/step - loss: 95438266368.0000 - mse: 95438266368.0000 - val_loss: 66462289920.0000 - val_mse: 66462289920.0000
Epoch 2/100
346/346 [==============================] - 1s 2ms/step - loss: 82706538496.0000 - mse: 82706538496.0000 - val_loss: 68789960704.0000 - val_mse: 68789960704.0000
Epoch 3/100
346/346 [==============================] - 1s 2ms/step - loss: 81174585344.0000 - mse: 81174585344.0000 - val_loss: 66461360128.0000 - val_mse: 66461360128.0000
Epoch 4/100
346/346 [==============================] - 1s 1ms/step - loss: 83067133952.0000 - mse: 83067133952.0000 - val_loss: 66938540032.0000 - val_mse: 66938540032.0000
Epoch 5/100
346/346 [==============================] - 1s 2ms/step - loss: 79018336256.0000 - mse: 79018336256.0000 - val_loss: 65944498176.0000 - val_mse: 65944498176.0000
Epoch 6/100
346/346 [==============================] - 1s 2ms/step - loss: 78927060992.0000 - mse: 78927060992.0000 - val_loss: 7941115

346/346 [==============================] - 1s 2ms/step - loss: 47411310592.0000 - mse: 47411310592.0000 - val_loss: 32975556608.0000 - val_mse: 32975556608.0000
Epoch 49/100
346/346 [==============================] - 1s 2ms/step - loss: 45425135616.0000 - mse: 45425135616.0000 - val_loss: 34466557952.0000 - val_mse: 34466557952.0000
Epoch 50/100
346/346 [==============================] - 1s 2ms/step - loss: 49171337216.0000 - mse: 49171337216.0000 - val_loss: 30441404416.0000 - val_mse: 30441404416.0000
Epoch 51/100
346/346 [==============================] - 1s 2ms/step - loss: 46364266496.0000 - mse: 46364266496.0000 - val_loss: 33946884096.0000 - val_mse: 33946884096.0000
Epoch 52/100
346/346 [==============================] - 1s 2ms/step - loss: 45544931328.0000 - mse: 45544931328.0000 - val_loss: 37014233088.0000 - val_mse: 37014233088.0000
Epoch 53/100
346/346 [==============================] - 1s 2ms/step - loss: 45030625280.0000 - mse: 45030625280.0000 - val_loss: 33416843264.00

346/346 [==============================] - 1s 2ms/step - loss: 41770934272.0000 - mse: 41770934272.0000 - val_loss: 31165087744.0000 - val_mse: 31165087744.0000
Epoch 96/100
346/346 [==============================] - 1s 1ms/step - loss: 42866753536.0000 - mse: 42866753536.0000 - val_loss: 30772740096.0000 - val_mse: 30772740096.0000
Epoch 97/100
346/346 [==============================] - 1s 2ms/step - loss: 39306608640.0000 - mse: 39306608640.0000 - val_loss: 26229755904.0000 - val_mse: 26229755904.0000
Epoch 98/100
346/346 [==============================] - 1s 2ms/step - loss: 43079274496.0000 - mse: 43079274496.0000 - val_loss: 30691158016.0000 - val_mse: 30691158016.0000
Epoch 99/100
346/346 [==============================] - 1s 2ms/step - loss: 38413029376.0000 - mse: 38413029376.0000 - val_loss: 79877668864.0000 - val_mse: 79877668864.0000
Epoch 100/100
346/346 [==============================] - 1s 2ms/step - loss: 40629387264.0000 - mse: 40629387264.0000 - val_loss: 28117567488.0

In [22]:
print('Evaluation at Epoch 100:\n')
for key in history_3.history:
    print(key + ':' + str(history_3.history[key][-1]) + '\n')
    
    
print('RMSE :' + str(np.sqrt(history_3.history['val_mse'][-1])))

Evaluation at Epoch 100:

loss:40629387264.0

mse:40629387264.0

val_loss:28117567488.0

val_mse:28117567488.0

RMSE :167682.937378852


This here gives us the lowest rmse in terms of validation scores. Compared to the other models, they all seem to perform better in terms of minimum error achieved. I probably have to do further tinkering of specific parameters to lower the error to compete with the other models.  